In [1]:
using LinearAlgebra

In [3]:
round(Int, sqrt(4))

2

In [5]:
fi = ComplexF32(1.0im)

0.0f0 + 1.0f0im

In [6]:
function vec2hermite(v::Vector{Float32})
    N = round(Int, sqrt(length(v)))
    H = zeros(ComplexF32, N, N)
    for i in 1:N
        for j in i:N
            l = N*(i-1) + 2j - i^2
            if(i == j)
                H[i,j] = v[l]
            else
                H[i,j] = v[l-1] + fi*v[l]
            end 
        end
    end
    H = Hermitian(H)
    return H
end

vec2hermite (generic function with 1 method)

In [10]:
function vec2unitary(v::Vector{Float32}, τ::Float32)
    H = vec2hermite(v)
    U = exp(fi*(τ*H))
    return U
end

vec2unitary (generic function with 1 method)

In [7]:
test = rand(Float32, 9)

9-element Vector{Float32}:
 0.579861
 0.3000999
 0.6829384
 0.23975629
 0.612435
 0.56028795
 0.16013771
 0.7568738
 0.6339585

In [12]:
H_test = vec2hermite(test)
U_test = vec2unitary(test, 0.1f0)

3×3 Matrix{ComplexF32}:
  0.993392+0.0576073im  -0.0720405+0.025441im   -0.0606471+0.0183969im
 0.0639517+0.0340373im    0.992743+0.0678701im  -0.0650247+0.0194355im
  0.061081+0.0295695im   0.0568746+0.0279734im    0.993361+0.0679102im

In [13]:
U_test*U_test'

3×3 Matrix{ComplexF32}:
         1.0+0.0im         -1.21072f-8+1.5134f-9im   7.45058f-9+3.72529f-9im
 -1.21072f-8-1.5134f-9im           1.0+0.0im         7.45058f-9-1.86265f-9im
  7.45058f-9-3.72529f-9im   7.45058f-9+1.86265f-9im         1.0+0.0im